In [2]:
import openai
import pandas as pd
from tqdm import tqdm
from dotenv import load_dotenv
import os
import random

# .env 파일에서 OpenAI API 키 로드
load_dotenv()
openai.api_key = os.environ.get("OPENAI_API_KEY")

# 카테고리 목록
categories = {
    '수유용품': ['젖병', '분유', '모유저장팩', '유축기', '수유쿠션', '수유패드', '젖병소독기', '수유등', '수유티', '젖꼭지', '젖병브러쉬', '분유케이스', '분유포트', '수유팔찌'],
    '이유식용품': ['이유식', '이유식마스터기', '이유식용기', '이유식냄비', '아기수저', '아기포크', '빨대컵', '이유식책', '이유식틀', '유아식판', '유아식탁보', '이유식가위', '이유식스푼', '실리콘턱받이'],
    '기저귀용품': ['기저귀', '물티슈', '기저귀케이크', '기저귀가방', '기저귀정리함', '기저귀갈이대', '기저귀크림', '엉덩이파우더', '기저귀털이', '기저귀스프레이', '천기저귀', '수영장기저귀'],
    '목욕스킨케어': ['베이비로션', '베이비오일', '아기샴푸', '아기욕조', '목욕타올', '아기면봉', '베이비파우더', '아기선크림', '아기비누', '아기bath', '목욕장난감', '아기샤워기', '아기목욕의자', '배꼽소독솜', '아기손톱가위', '아기빗', '아기면봉', '아기귀걸이'],
    '수면용품': ['아기침대', '유아베개', '스와들', '아기이불', '모빌', '백색소음기', '수면조끼', '아기침대가드', '유아베개', '아기침구세트', '아기요', '아기매트리스', '아기침대모기장', '수유쿠션'],
    '의류': ['아기옷', '바디수트', '아기내복', '아기양말', '아기모자', '턱받이', '속싸개', '아기외출복', '아기신발', '아기장갑', '아기우비', '아기수영복', '아기한복', '아기카디건', '아기조끼', '아기잠옷'],
    '외출용품': ['유모차', '아기띠', '카시트', '아기가방', '보행기', '아기썬캡', '아기선글라스', '아기슬링', '유모차정리함', '유모차레인커버', '아기모기장', '아기미아방지용품', '아기보냉백'],
    '놀이교육용품': ['장난감', '소리나는책', '블록', '모빌', '치발기', '아기체육관', '아기책', '아기퍼즐', '아기그림책', '아기악기', '아기놀이매트', '아기워커', '아기그네', '아기점프', '아기볼풀', '미술놀이세트', '아기칠판'],
    '안전용품': ['베이비모니터', '콘센트커버', '모서리보호대', '안전문', '미끄럼방지매트', '아기안전경보기', '유아안전벨트', '아기안전가드', '아기안전잠금장치', '아기차량용품', '유아헬멧'],
    '위생용품': ['손소독제', '마스크', '체온계', '콧물흡입기', '아기치약', '아기칫솔', '네일케어세트', '귀체온계', '아기면봉', '아기손수건', '살균소독기', '공기청정기'],
    '기타육아용품': ['쪽쪽이', '떡뻥', '아기의자', '아기식탁의자', '아기체중계', '아기발달체조', '아기학습테이블', '유아변기', '유아세면대', '아기욕실의자', '아기발육알림판'],
    '부모용품': ['수유브라', '수유복', '산모패드', '산모벨트', '산후복대', '유축기가방', '산모영양제', '산모방석', '산모슬리퍼', '산모클렌징', '산모마사지크림', '산모좌욕기'],
    '신생아용품': ['배냇저고리', '손싸개', '발싸개', '배띠', '가제수건', '신생아모자', '신생아손발싸개', '탯줄관리용품', '신생아내복']
}

# 태그 목록
tags = [
    "아기 용품"
]

# 조건 설정
ages = ["20대 후반", "30대 초반", "30대 후반"]
experiences = ["초보", "둘째를 키우는"]
genders = ["엄마", "아빠"]
child_ages = ["0세", "1세", "2세"]
seasons = ["봄", "여름", "가을", "겨울"]
special_situations = ["일반", "쌍둥이", "장애아", "다문화 가정"]

# 비율 설정
age_ratio = [1, 1, 1]
experience_ratio = [1, 1]
gender_ratio = [6, 4]
child_age_ratio = [1, 1, 1]
season_ratio = [1, 1, 1, 1]
special_situation_ratio = [7, 1, 1, 1]

def select_weighted_random(choices, weights):
    return random.choices(choices, weights, k=1)[0]

def generate_prompt(category, tag, age, experience, gender, child_age, season, special_situation, item):
    base_prompt = (
        f"{age} {experience} {gender}가 {child_age} 자녀를 키우고 있습니다. "
        f"현재 계절은 {season}입니다. "
        f"태그는 {tag}이며, "
        f"질문은 50자 내외로 작성하고, 최근에 겪은 특정 상황이나 문제점을 포함하며, "
        f"실제 부모가 사용할 법한 자연스러운 표현으로 작성해주세요. "
    )

    item_prompt = (
        f"{category}에 관한 질문이야. '{item}'에 대해 물어보는 질문을 만들어주세요. "
        f"질문에는 {category}, {item}, 그리고 그 item에 대한 관련 정보가 반드시 포함되어야 해."
        f"예를 들어, '수유용품에 관한 질문이야. 아이에게 주는 젖병을 소독하기 위해서 쓰는 건 젖병소독기가 맞아?'와 같은 형태로 작성해주세요."
    )
    
    return base_prompt + item_prompt

# 데이터 프레임 초기화
data = []

# 각 카테고리에서 균일하게 질문 생성
total_questions = 100
questions_per_category = total_questions // len(categories)
remaining_questions = total_questions % len(categories)

for category, items in categories.items():
    num_questions = questions_per_category + (1 if remaining_questions > 0 else 0)
    remaining_questions -= 1 if remaining_questions > 0 else 0

    for _ in range(num_questions):
        item = random.choice(items)
        tag = random.choice(tags)
        age = select_weighted_random(ages, age_ratio)
        experience = select_weighted_random(experiences, experience_ratio)
        gender = select_weighted_random(genders, gender_ratio)
        child_age = select_weighted_random(child_ages, child_age_ratio)
        season = select_weighted_random(seasons, season_ratio)
        special_situation = select_weighted_random(special_situations, special_situation_ratio)

        prompt = generate_prompt(category, tag, age, experience, gender, child_age, season, special_situation, item)
        
        # GPT-3.5-turbo API 호출
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": "You are a helpful assistant."},
                {"role": "user", "content": prompt}
            ],
            max_tokens=100
        )
        
        # 생성된 질문 추출
        question = response['choices'][0]['message']['content'].strip()
        
        # 데이터 추가
        data.append({
            "category": category,
            "tag": tag,
            "age": age,
            "experience": experience,
            "gender": gender,
            "child_age": child_age,
            "season": season,
            "special_situation": special_situation,
            "item": item,
            "question": question
        })

# 데이터 프레임 생성
df = pd.DataFrame(data)

# CSV 파일로 저장
df.to_csv("generated_questions.csv", index=False)

print("Questions generation complete. Saved to 'generated_questions.csv'.")

Questions generation complete. Saved to 'generated_questions.csv'.


In [1]:
import openai
import pandas as pd
from tqdm import tqdm
from dotenv import load_dotenv
import os
import random

# .env 파일에서 OpenAI API 키 로드
load_dotenv()
openai.api_key = os.environ.get("OPENAI_API_KEY")

In [2]:
# 태그 목록
tags = [
    "병원", "육아 상담"
]

# 조건 설정
ages = ["20대 후반", "30대 초반", "30대 후반"]
experiences = ["초보", "둘째를 키우는"]
genders = ["엄마", "아빠"]
child_ages = ["0세", "1세", "2세"]
seasons = ["봄", "여름", "가을", "겨울"]
special_situations = ["일반", "쌍둥이", "장애아", "다문화 가정"]

# 비율 설정
age_ratio = [1, 1, 1]
experience_ratio = [1, 1]
gender_ratio = [6, 4]
child_age_ratio = [1, 1, 1]
season_ratio = [1, 1, 1, 1]
special_situation_ratio = [7, 1, 1, 1]

def generate_prompt(tag, age, experience, gender, child_age, season, special_situation):
    base_prompt = (
        f"{age} {experience} {gender}가 {child_age} 자녀를 키우고 있습니다. "
        f"현재 계절은 {season}입니다. "
    )
    
    if special_situation != "일반":
        base_prompt += f"이 가정은 {special_situation} 상황입니다. "
    
    if tag == "병원":
        base_prompt += (
            f"{child_age} 자녀를 키우는 {gender}의 입장에서, 병원 위치, 병원 정보, "
            f"또는 급한 병원 방문이 필요한 상황에 대한 질문을 하나 생성해주세요. "
            f"질문에는 반드시 '병원' 키워드가 포함되어야 합니다. "
            f"육아 상담 태그와 헷갈리지 않는 질문이어야 합니다."
        )
    elif tag == "육아 상담":
        base_prompt += (
            f"{child_age} 자녀를 키우는 {gender}의 입장에서, 아이의 행동이나 발달에 대한 조언을 구하는 질문을 하나 생성해주세요. "
            f"질문은 급하지 않은 일상적인 육아 고민이어야 합니다. "
            f"병원 태그와 헷갈리지 않는 질문이어야 합니다."
        )
    
    base_prompt += (
        f"질문은 30자 내외로 작성하고, 최근에 겪은 특정 상황이나 문제점을 포함하며, "
        f"실제 부모가 사용할 법한 자연스러운 표현으로 작성해주세요. "
    )
    
    return base_prompt

In [3]:
# 데이터 프레임 초기화
data = []

# 각 태그별로 100개의 질문 생성
for tag in tags:
    for _ in tqdm(range(100), desc=f"Generating questions for tag: {tag}"):
        while True:
            # 각 비율에 따라 무작위 선택
            age = random.choices(ages, weights=age_ratio, k=1)[0]
            experience = random.choices(experiences, weights=experience_ratio, k=1)[0]
            gender = random.choices(genders, weights=gender_ratio, k=1)[0]
            child_age = random.choices(child_ages, weights=child_age_ratio, k=1)[0]
            season = random.choices(seasons, weights=season_ratio, k=1)[0]
            special_situation = random.choices(special_situations, weights=special_situation_ratio, k=1)[0]
            
            # 프롬프트 생성
            prompt = generate_prompt(tag, age, experience, gender, child_age, season, special_situation)
            
            # GPT-3.5-turbo API 호출
            response = openai.ChatCompletion.create(
                model="gpt-3.5-turbo",
                messages=[
                    {"role": "system", "content": "You are a helpful assistant."},
                    {"role": "user", "content": prompt}
                ],
                max_tokens=100
            )
            
            # 생성된 질문 추출
            question = response['choices'][0]['message']['content'].strip()
            
            # 태그에 맞는 질문인지 확인
            if tag == "병원" and "병원" in question:
                break
            elif tag == "육아 상담" and "병원" not in question:
                break
        
        # 데이터 추가
        data.append({
            "tag": tag,
            "age": age,
            "experience": experience,
            "gender": gender,
            "child_age": child_age,
            "season": season,
            "special_situation": special_situation,
            "question": question
        })

# 데이터 프레임 생성
df = pd.DataFrame(data)

# CSV 파일로 저장
df.to_csv("generated_questions.csv", index=False)

print("Questions generation complete. Saved to 'generated_questions.csv'.")

Generating questions for tag: 병원:   0%|          | 0/100 [00:00<?, ?it/s]

Generating questions for tag: 육아 상담: 100%|██████████| 100/100 [01:51<00:00,  1.12s/it]

Questions generation complete. Saved to 'generated_questions.csv'.


In [3]:
import pandas as pd
import random

# processed_data.csv에서 '아기 용품' 태그 삭제
processed_data = pd.read_csv('processed_data.csv')
processed_data = processed_data[~processed_data['tag'].isin(['아기 용품'])]

# aa.csv에서 '아기 용품' 태그 행 추출
qq_data = pd.read_csv('aa.csv')
qq_filtered = qq_data[qq_data['tag'] == '아기 용품']


# 최종 데이터프레임 합치기
final_data = pd.concat([processed_data, qq_filtered[['tag', 'question']]], ignore_index=True)

# CSV 파일로 저장
final_data.to_csv('merged_data.csv', index=False)

print("CSV 파일이 성공적으로 생성되었습니다. 'merged_data.csv' 파일을 확인해주세요.")


CSV 파일이 성공적으로 생성되었습니다. 'merged_data.csv' 파일을 확인해주세요.
